In [3]:
import numpy as np
import pandas as pd
import cvxpy as cp
import gym
import matplotlib.pyplot as plt
from numpy.linalg import LinAlgError


import sys
sys.path.append('./asebo/')
from optimizers import Adam
from worker import worker, get_policy
from es import ES

In [22]:
def F(theta):
    if theta.ndim == 1:
        theta = np.expand_dims(theta, 0)
    return -np.sum((theta - 0.5) ** 2 , axis=tuple(range(theta.ndim)[1:]))\
           -np.sum(theta[:, 1:]*theta[:, :-1], axis=tuple(range(theta.ndim)[1:]))


def Gradient_LP(y, epsilons):
    """
    y = (F(theta + sigma*epsilons) - F(theta)) / sigma
    epsilons: the perturbations with UNIT VARIANCE
    """
    n, d = epsilons.shape
    
    var_z = cp.Variable(n)
    var_g = cp.Variable(d)
    obj = sum(var_z)
    constraints = [var_z >= y - epsilons @ var_g,
                   var_z >= -y + epsilons @ var_g]
    prob = cp.Problem(cp.Minimize(obj), constraints)
    prob.solve(solver=cp.GLPK, eps=1e-6, glpk={'msg_lev': 'GLP_MSG_OFF'})
    if prob.status == 'optimal':
        return var_g.value
    return None


def Hessian_LP(y, epsilons):
    """
    y = (F(theta + sigma*epsilons) + F(theta - sigma*epsilons) - 2*F(theta)) / (sigma**2)
    epsilons: the perturbations with UNIT VARIANCE
    """
    n, d = epsilons.shape
    
    
    X = np.zeros((n, d*(d+1)//2))
    idx = 0
    for j in range(d):
        X[:,idx] = epsilons[:,j]**2
        idx += 1
        if j == d-1:
            break
        X[:,idx:idx+d-j-1] = 2 * epsilons[:,j:j+1] * epsilons[:,j+1:]
        idx += d-j-1
#         X[:,j*(j+1)//2:(j+1)*(j+2)//2-1] = 2 * epsilons[:,j:j+1] * epsilons[:,:j]
#         X[:,(j+1)*(j+2)//2-1] = epsilons[:,j]**2
    
    var_z = cp.Variable(n)
    var_H = cp.Variable(d*(d+1)//2)
    
    obj = sum(var_z)
    
    constraints = []
    for i in range(n):
        constraints += [var_z[i] >= y[i] - X[i] @ var_H]
        constraints += [var_z[i] >= - y[i] + X[i] @ var_H]
    
    prob = cp.Problem(cp.Minimize(obj), constraints)
    prob.solve(solver=cp.GLPK, eps=1e-6, glpk={'msg_lev': 'GLP_MSG_OFF'})

    if prob.status == 'optimal':
        H = np.zeros((d,d))
        idx = 0
        for j in range(d):
            H[j,j:] = var_H[idx:idx+d-j].value
            H[j:,j] = var_H[idx:idx+d-j].value
            idx += d-j
#             H[j,0:j+1] = var_H[j*(j+1)//2:(j+1)*(j+2)//2].value
#             H[1:j+1,j] = H[j,1:j+1]
        return H
    return None

## Manuel Tests

# Testing

In [33]:
sigma = 0.05
# np.random.seed(0)
theta = np.random.uniform(-5,5,5)
# theta = 0.0 * np.ones(5)
n = 100
d = len(theta)

print('theta:')
print(theta)
epsilons = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d), size = n)

gradient_y = (F(theta + sigma*epsilons) - F(theta)) / sigma
hessian_y =(F(theta + sigma*epsilons) + F(theta - sigma*epsilons) - 2*F(theta)) / (sigma**2)
g = Gradient_LP(gradient_y, epsilons)
print('gradient:')
print(g)

H = Hessian_LP(hessian_y, epsilons)
print('Hessian:')
print(H)

theta:
[ 4.8058753   1.32000452 -4.05692195  0.09458213  0.01032006]
gradient:
[-9.8934653  -2.31129183  7.7322296   4.98261804  0.92235917]
Hessian:
[[-2.00000000e+00 -1.00000000e+00  2.23416014e-13 -4.29916749e-13
   3.84345008e-13]
 [-1.00000000e+00 -2.00000000e+00 -1.00000000e+00  1.31117394e-13
   8.86488843e-14]
 [ 2.23416014e-13 -1.00000000e+00 -2.00000000e+00 -1.00000000e+00
   1.90527724e-13]
 [-4.29916749e-13  1.31117394e-13 -1.00000000e+00 -2.00000000e+00
  -1.00000000e+00]
 [ 3.84345008e-13  8.86488843e-14  1.90527724e-13 -1.00000000e+00
  -2.00000000e+00]]


In [25]:

def get_dct_mtx(d):
    # DCT matrix
    # unitary, symmetric and real
    # Orthonormal eigenbasis for structured H
    n = 2*d
    i_idx = np.array([range(n//2 )])
    idx = 2 * np.transpose(i_idx) @ i_idx
    dct_mtx = np.cos(idx*np.pi / n) * 2 / np.sqrt(d)
    dct_mtx[0,0] = 1
    dct_mtx[0,d-1] = 1
    dct_mtx[d-1,0] = 1
    dct_mtx[d-1,d-1] = (-1)**(d)
    return dct_mtx
def Hessian_LP_structured(sigma, theta, num_samples):
    # LP formulation to estimate Hessian
    # Minimizing over the space of matrices of the form
    # shown in the example 7 & 8 in the reference
    # [MATRICES DIAGONALIZED BY THE DISCRETECOSINE AND DISCRETE SINE TRANSFORMS]
    
    d = len(theta)
    n = num_samples
    
    epsilons = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d), size = n)
    
    # Define and solve the LP for Hessian here
    y = (F(theta + sigma * epsilons) + F(theta - sigma * epsilons) - 2 * F(theta)) / (sigma ** 2)
    
    var_z = cp.Variable(n)
    var_H_diag = cp.Variable(d)
    
    # Lower triangular mtx H
    dct_mtx = get_dct_mtx(d)
    obj = sum(var_z)
    
    constraints = []
    for i in range(n):
        Uv = epsilons[i:i+1,:] @ dct_mtx
        Uv_sq = Uv * Uv
        constraints += [var_z[i] >= y[i] - Uv_sq @ var_H_diag]
        constraints += [var_z[i] >= - y[i] + Uv_sq @ var_H_diag]
    for i in range(d):
        constraints += [var_H_diag[i] <= 0]
        
    prob = cp.Problem(cp.Minimize(obj), constraints)
    prob.solve(solver=cp.GLPK, eps=1e-6, glpk={'msg_lev': 'GLP_MSG_OFF'})
    
    # 
    if prob.status == 'optimal':
        return dct_mtx @ np.diag(var_H_diag.value) @ np.transpose(dct_mtx)
    
    return None

In [30]:
sigma = 0.05
theta = np.random.uniform(-5,5,5)
Hessian_LP_structured(sigma, theta, 1000)

array([[-3.39250461, -1.41592579, -0.0216422 , -0.26101228, -0.17156035],
       [-1.41592579, -1.55673325, -0.76789389, -0.01985786, -0.31046238],
       [-0.0216422 , -0.76789389, -1.63957978, -0.77185871, -0.07637728],
       [-0.26101228, -0.01985786, -0.77185871, -1.63957978, -0.82441331],
       [-0.17156035, -0.31046238, -0.07637728, -0.82441331, -1.84637941]])